In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from __future__ import print_function

from matplotlib import rcParams
rcParams["savefig.dpi"] = 200
rcParams["font.size"] = 8

import warnings
warnings.filterwarnings('ignore')

# Multidimensional coordinates

## Multidimensional coordinates with xarray

Regionmask can also handle mutltidimensional longitude/ latitude grids (e.g. from a regional climate model). As xarray provides such an example dataset, we will use it to illustrate it. See also in the [xarray documentation](http://xarray.pydata.org/en/stable/examples/multidimensional-coords.html/).


Import regionmask and check the version:

In [ ]:
import regionmask
regionmask.__version__

Load xarray and the tutorial data:

In [ ]:
import xarray as xr
import numpy as np

In [ ]:
rasm = xr.tutorial.load_dataset('rasm')

The example data is a temperature field over the Northern Hemisphere. Let's plot the first time step:

In [ ]:
# load plotting libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# choose a projection
proj=ccrs.NorthPolarStereo()

ax = plt.subplot(111, projection=proj)
ax.set_global()

rasm.isel(time=1).Tair.plot.pcolormesh(ax=ax, x='xc', y='yc', transform=ccrs.PlateCarree())

# add the abbreviation of the regions
regionmask.defined_regions.srex.plot(ax=ax, regions=[1, 2, 11, 12, 18], 
                                     add_ocean=False, coastlines=False, label='abbrev')

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())

ax.coastlines();

Again we pass the xarray object to regionmask. We have to specify `xc` and `yc` as the longitude and latitude coordinates of the array: 

In [ ]:
mask = regionmask.defined_regions.srex.mask(rasm, lon_name='xc', lat_name='yc', wrap_lon=True)
mask

We want to select the region 'NAS' (Northern Asia).

### Select using `where`

We have to select by index (the number of the region), we thus map from the abbreviation to the index.

In [ ]:
rasm_NAS = rasm.where(mask == regionmask.defined_regions.srex.map_keys('NAS'))

Check everything went well by repeating the first plot with the selected region:

In [ ]:
# choose a projection
proj=ccrs.NorthPolarStereo()

ax = plt.subplot(111, projection=proj)
ax.set_global()

rasm_NAS.isel(time=1).Tair.plot.pcolormesh(ax=ax, x='xc', y='yc', transform=ccrs.PlateCarree())


# add the abbreviation of the regions
regionmask.defined_regions.srex.plot(ax=ax, regions=['NAS'], 
                                     add_ocean=False, coastlines=False, label='abbrev')

ax.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())

ax.coastlines();

Looks good - now we coould again take the area average.

## Multidimensional coordinates with numpy

If you don't use xarray, the same can be achieved with numpy arrays.

In [ ]:
# extract lon and lat
lon = rasm.xc.values
lat = rasm.yc.values

The lon and lat numpy arrays can directly be passed to regionmask (now we create a mask with the countries of the world):

In [ ]:
mask = regionmask.defined_regions.natural_earth.countries_110.mask(lon, lat, wrap_lon=True)
mask

If xarray is installed, regionmask automatically returns a xarray object even when passing numpy arrays. To get a numpy array, use the `xarray=False` keyword.

In [ ]:
# mask = regionmask.defined_regions.natural_earth.countries_110.mask(lon, lat, xarray=False, wrap_lon=True)

Or just use `mask.values`:

In [ ]:
mask = mask.values
mask

Finally, plot the mask:

In [ ]:
proj=ccrs.NorthPolarStereo()

ax = plt.subplot(111, projection=proj)

m = np.ma.masked_invalid(mask)
ax.pcolormesh(lon, lat, m, transform=ccrs.PlateCarree())

ax.set_extent([-180, 180, 45, 90], ccrs.PlateCarree())

ax.coastlines();